### Code for Feature Analysis

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import KNNImputer


account_info_df = pd.read_csv('data/account_dim_20250325.csv')
account_info_df['open_date'] = pd.to_datetime(account_info_df['open_date'], errors='coerce')

transaction_df = pd.read_csv('data/transaction_fact_20250325.csv')
statement_df = pd.read_csv('data/statement_fact_20250325.csv')
fraud_case_df = pd.read_csv('data/fraud_claim_case_20250325.csv')
fraud_transaction_df = pd.read_csv('data/fraud_claim_tran_20250325.csv')
world_transaction_df = pd.read_csv('data/wrld_stor_tran_fact_20250325.csv')
account_features_df = pd.read_csv('data/rams_batch_cur_20250325.csv')

transaction_df['transaction_date'] = pd.to_datetime(transaction_df['transaction_date'])
fraud_case_df['reported_date'] = pd.to_datetime(fraud_case_df['reported_date'])

all_transactions = pd.concat([
    transaction_df[['current_account_nbr', 'transaction_date', 'transaction_amt', 'transaction_type']],
    world_transaction_df[['current_account_nbr', 'transaction_date', 'transaction_amt', 'transaction_type']]
])


transaction_pivot = all_transactions.pivot_table(
    index='current_account_nbr', 
    columns=['transaction_date', 'transaction_type'], 
    values='transaction_amt', 
    aggfunc='sum'
)


transaction_pivot.columns = [f"{date}_{type}" for date, type in transaction_pivot.columns]


imputer = KNNImputer(n_neighbors=5)
transaction_pivot_imputed = pd.DataFrame(
    imputer.fit_transform(transaction_pivot),
    index=transaction_pivot.index,
    columns=transaction_pivot.columns
)


all_transactions_imputed = transaction_pivot_imputed.stack().reset_index()
all_transactions_imputed[['transaction_date', 'transaction_type']] = all_transactions_imputed['level_1'].str.split('_', expand=True)

all_transactions_imputed.columns = ['current_account_nbr', 'transaction_date', 'transaction_amt', 'transaction_type']
all_transactions_imputed['quarter'] = pd.to_datetime(all_transactions_imputed['transaction_date'], format='ISO8601').dt.to_period('Q')
all_transactions_imputed['month'] = pd.to_datetime(all_transactions_imputed['transaction_date'], format='ISO8601').dt.to_period('M')
all_transactions_imputed['transaction_amt'] = all_transactions_imputed[0]
all_transactions_imputed.drop(columns=['level_1', 0], inplace=True)
all_transactions_imputed.to_csv("temp_all_trans.csv")
spending_behavior = all_transactions_imputed.groupby(['current_account_nbr', 'quarter', 'transaction_type'])['transaction_amt'].agg(
    total_spending='sum',
    avg_spending='mean',
    std_spending='std'
).reset_index()

quarters = spending_behavior['quarter'].astype(str).unique()
spending_pivot = pd.DataFrame(index=account_info_df['current_account_nbr'].unique())
# Create a pivot table with all the metrics at once
spending_pivot = spending_behavior.pivot_table(
    index='current_account_nbr',
    columns=['quarter', 'transaction_type'],
    values=['total_spending', 'avg_spending', 'std_spending']
)

# Flatten the column names
spending_pivot.columns = [f"{quarter}_{transaction_type}_{metric}" 
                         for (metric, quarter, transaction_type) in spending_pivot.columns]

spending_pivot.reset_index(inplace=True)
spending_pivot.rename(columns={'index': 'current_account_nbr'}, inplace=True)

# First, identify the unique quarters and transaction types
quarters = sorted(list(set([col.split('_')[0] for col in spending_pivot.columns if '_total_spending' in col])))
transaction_types = sorted(list(set([col.split('_')[1] for col in spending_pivot.columns if '_total_spending' in col])))

# Calculate velocity for each transaction type
for i in range(len(quarters)-1):
    q1 = quarters[i]
    q2 = quarters[i+1]
    
    for trans_type in transaction_types:
        q1_col = f'{q1}_{trans_type}_total_spending'
        q2_col = f'{q2}_{trans_type}_total_spending'
        
        if q1_col in spending_pivot.columns and q2_col in spending_pivot.columns:
            spending_pivot[f'{q1}_{q2}_{trans_type}_velocity'] = (
                spending_pivot[q2_col] - spending_pivot[q1_col]
            )
        else:
            print(f"Skipping velocity calculation for {q1} to {q2} ({trans_type}) - columns not found")
    
    # Calculate transaction frequency
    q1_transactions = all_transactions_imputed[all_transactions_imputed['quarter'].astype(str) == q1]
    q1_freq = q1_transactions.groupby('current_account_nbr').size()
    spending_pivot[f'{q1}_transaction_frequency'] = q1_freq.reindex(spending_pivot.index).fillna(0)

# Calculate acceleration for each transaction type
for i in range(len(quarters)-2):
    q1 = quarters[i]
    q2 = quarters[i+1]
    q3 = quarters[i+2]
    
    for trans_type in transaction_types:
        v1_col = f'{q1}_{q2}_{trans_type}_velocity'
        v2_col = f'{q2}_{q3}_{trans_type}_velocity'
        
        if v1_col in spending_pivot.columns and v2_col in spending_pivot.columns:
            spending_pivot[f'{q1}_{q2}_{q3}_{trans_type}_acceleration'] = (
                spending_pivot[v2_col] - spending_pivot[v1_col]
            )
        else:
            print(f"Skipping acceleration calculation for {q1} to {q3} ({trans_type}) - velocity columns not found")

# Optionally, calculate overall velocity and acceleration (across all transaction types)
for i in range(len(quarters)-1):
    q1 = quarters[i]
    q2 = quarters[i+1]
    
    spending_pivot[f'{q1}_{q2}_overall_velocity'] = sum(
        spending_pivot.get(f'{q1}_{q2}_{trans_type}_velocity', 0) 
        for trans_type in transaction_types
    )

for i in range(len(quarters)-2):
    q1 = quarters[i]
    q2 = quarters[i+1]
    q3 = quarters[i+2]
    
    spending_pivot[f'{q1}_{q2}_{q3}_overall_acceleration'] = (
        spending_pivot[f'{q2}_{q3}_overall_velocity'] - spending_pivot[f'{q1}_{q2}_overall_velocity']
    )

account_info_df['open_date'] = pd.to_datetime(account_info_df['open_date'], errors='coerce')
account_info_df['open_date_naive'] = account_info_df['open_date'].dt.tz_localize(None)
account_info_df['age_days'] = (datetime(2025, 3, 29) - account_info_df['open_date_naive']).dt.days

activation_map = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
account_info_df['activation_status_numeric'] = account_info_df['card_activation_flag'].map(activation_map)

ebill_map = {'B': 1, 'E': 2, 'L': 3, ' ': 0, '': 0}
account_info_df['ebill_ind_numeric'] = account_info_df['ebill_ind'].map(ebill_map)

overlimit_map = {'0': 0, '1': 1, '2': 2, '3': 3}
account_info_df['overlimit_type_flag_numeric'] = account_info_df['overlimit_type_flag'].map(overlimit_map)

employee_map = {'0': 0, '1': 1, 'H': 2}
account_info_df['employee_code_numeric'] = account_info_df['employee_code'].map(employee_map)

finance_map = {'0': 0, '1': 1}
account_info_df['special_finance_charge_numeric'] = account_info_df['special_finance_charge_ind'].map(finance_map)

account_info_df['pscc_ind_numeric'] = account_info_df['pscc_ind'].astype(int)

def analyze_payment_history(payment_hist):
    if not isinstance(payment_hist, str):
        return 0, 0, 0
    
    delinquency_count = sum(c in 'bcdefghjklmnopuvwx1234567' for c in payment_hist.lower())
    zero_balance_count = payment_hist.lower().count('z')
    overlimit_count = payment_hist.lower().count('o')
    
    return delinquency_count, zero_balance_count, overlimit_count

payment_features = statement_df.apply(
    lambda row: analyze_payment_history(row['payment_hist_1_12_mths']), 
    axis=1, 
    result_type='expand'
)
payment_features.columns = ['delinquency_count', 'zero_balance_count', 'overlimit_count']
statement_df = pd.concat([statement_df, payment_features], axis=1)

payment_summary = statement_df.groupby('current_account_nbr').agg(
    delinquency_count=('delinquency_count', 'sum'),
    zero_balance_count=('zero_balance_count', 'sum'),
    overlimit_count=('overlimit_count', 'sum'),
    return_check_count_total=('return_check_cnt_total', 'max')
).reset_index()


fraud_case_summary = fraud_case_df.groupby('current_account_nbr').agg(
    gross_fraud_amt=('gross_fraud_amt', 'sum'),
    net_fraud_amt=('net_fraud_amt', 'sum'),
    fraud_frequency=('case_id', 'count')
).reset_index()

fraud_case_summary['fraud_ratio'] = fraud_case_summary['net_fraud_amt'] / fraud_case_summary['gross_fraud_amt']
fraud_codes = {13, 22, 23, 46, 48, 62, 80}

if 'external_status_reason_code' in account_info_df.columns:
    account_info_df['external_status_numeric'] = account_info_df['external_status_reason_code'].apply(lambda x: 1 if x in fraud_codes else 0)

account_features_df = account_features_df.rename(columns={
    'cu_account_nbr': 'current_account_nbr',
    'ca_current_utilz': 'utilization_rate',
    'cu_otb': 'open_to_buy',
    'cu_cur_balance': 'current_balance'
})

latest_transaction = all_transactions_imputed.groupby('current_account_nbr')['transaction_date'].max().reset_index()
latest_transaction.columns = ['current_account_nbr', 'last_transaction_date']
latest_transaction['days_since_last_transaction'] = (
    datetime(2025, 3, 29) - pd.to_datetime(latest_transaction['last_transaction_date'])
).dt.days

account_features_df['max_days_delinquent'] = account_features_df['cu_nbr_days_dlq']
account_features_df['current_cycles_delinquent'] = account_features_df['cu_cur_nbr_due']

account_features_df['balance_to_credit_ratio'] = account_features_df['current_balance'] / account_features_df['cu_crd_line']
account_features_df['cash_balance_to_cash_line_ratio'] = account_features_df['ca_cash_bal_pct_cash_line'] / 100

credit_grade_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
account_features_df['credit_grade_numeric'] = account_features_df['rb_crd_gr_new_crd_gr'].map(credit_grade_map)

account_features_df['months_since_active'] = account_features_df['ca_mnths_since_active']
account_features_df['months_since_credit_limit_change'] = account_features_df['ca_mnths_since_cl_chng']
account_features_df['months_on_books'] = account_features_df['ca_mob']

account_features_df['utilization_stability'] = account_features_df['ca_avg_utilz_lst_3_mnths'] / account_features_df['ca_avg_utilz_lst_6_mnths']
account_features_df['financial_stress_indicator'] = account_features_df['open_to_buy'] / account_features_df['cu_crd_line']

monthly_sales_columns = [f'mo_tot_sales_array_{i}' for i in range(1, 7)]
account_features_df['total_sales_last_6_months'] = account_features_df[monthly_sales_columns].sum(axis=1)
account_features_df['avg_monthly_sales_last_6_months'] = account_features_df[monthly_sales_columns].mean(axis=1)

account_features_df['sales_trend'] = (account_features_df['mo_tot_sales_array_1'] - 
                                      account_features_df['mo_tot_sales_array_6']) / 6

account_features_df['nsf_count_last_12_months'] = account_features_df['ca_nsf_count_lst_12_months']

account_features_df['behavior_score_change'] = account_features_df['rb_new_bhv_scr'] - account_features_df['cu_bhv_scr']
account_features_df['behavior_score_percent_change'] = (account_features_df['behavior_score_change'] / 
                                                       account_features_df['cu_bhv_scr']) * 100

account_features_df['high_delinquency_flag'] = (account_features_df['cu_nbr_days_dlq'] > 60).astype(int)
account_features_df['rapid_balance_change_flag'] = (
    (account_features_df['ca_avg_utilz_lst_3_mnths'] - account_features_df['ca_avg_utilz_lst_6_mnths']) > 20
).astype(int)

account_features_df['fraud_risk_score'] = (
    account_features_df['high_delinquency_flag'] * 3 + 
    account_features_df['rapid_balance_change_flag'] * 2 +
    (account_features_df['months_since_active'] < 3).astype(int) * 2 +
    (account_features_df['behavior_score_percent_change'] < -10).astype(int) * 3 +
    account_features_df['credit_grade_numeric'] * 5
    
)

for i in range(1, 6):
    account_features_df[f'sales_change_month_{i}_to_{i+1}'] = (
        account_features_df[f'mo_tot_sales_array_{i}'] - account_features_df[f'mo_tot_sales_array_{i+1}']
    )

account_features_df['sales_volatility'] = account_features_df[monthly_sales_columns].std(axis=1)



total_spending = all_transactions_imputed.groupby('current_account_nbr')['transaction_amt'].sum().reset_index()
account_features_df = account_features_df.merge(total_spending, on='current_account_nbr', how='left')
account_features_df['transaction_amt'] = account_features_df['transaction_amt'].fillna(0)
account_features_df['cash_balance'] = account_features_df['cu_crd_line'] - account_features_df['transaction_amt']



account_features_df['negative_cash_balance_flag'] = (account_features_df['cash_balance'] < 0).astype(int)

monthly_cash_balances = all_transactions_imputed.groupby(['current_account_nbr', 'month'])['transaction_amt'].sum().reset_index()


monthly_cash_balances = monthly_cash_balances.sort_values(['current_account_nbr', 'month'])
monthly_cash_balances['running_cash_balance'] = monthly_cash_balances.groupby('current_account_nbr')['transaction_amt'].cumsum()


cash_volatility = monthly_cash_balances.groupby('current_account_nbr')['running_cash_balance'].std().reset_index()
cash_volatility.columns = ['current_account_nbr', 'cash_balance_volatility']
account_features_df = account_features_df.merge(cash_volatility, on='current_account_nbr', how='left')
account_features_df['cash_balance_volatility'] = account_features_df['cash_balance_volatility'].fillna(0)


account_features_df['cash_flow_efficiency'] = account_features_df['cash_balance'] / account_features_df['cu_crd_line']
account_features_df['cash_flow_efficiency'] = account_features_df['cash_flow_efficiency'].clip(lower=-1, upper=1)



monthly_trends = monthly_cash_balances.groupby('current_account_nbr').apply(
    lambda x: np.polyfit(range(len(x)), x['running_cash_balance'], 1)[0] if len(x) > 1 else 0
).reset_index()
monthly_trends.columns = ['current_account_nbr', 'cash_balance_trend']
account_features_df = account_features_df.merge(monthly_trends, on='current_account_nbr', how='left')
account_features_df['cash_balance_trend'] = account_features_df['cash_balance_trend'].fillna(0)


account_features_df['cash_to_debt_ratio'] = account_features_df['cash_balance'] / account_features_df['current_balance']
account_features_df['cash_to_debt_ratio'] = account_features_df['cash_to_debt_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)


account_features_df['cash_buffer_ratio'] = account_features_df['cash_balance'] / account_features_df['avg_monthly_sales_last_6_months']
account_features_df['cash_buffer_ratio'] = account_features_df['cash_buffer_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)


negative_accounts = monthly_cash_balances[monthly_cash_balances['running_cash_balance'] < 0]
negative_frequency = negative_accounts.groupby('current_account_nbr').size().reset_index()
negative_frequency.columns = ['current_account_nbr', 'negative_cash_balance_frequency']

negative_magnitude = negative_accounts.groupby('current_account_nbr')['running_cash_balance'].mean().reset_index()
negative_magnitude.columns = ['current_account_nbr', 'average_negative_cash_balance']

account_features_df = account_features_df.merge(negative_frequency, on='current_account_nbr', how='left')
account_features_df = account_features_df.merge(negative_magnitude, on='current_account_nbr', how='left')
account_features_df['negative_cash_balance_frequency'] = account_features_df['negative_cash_balance_frequency'].fillna(0)
account_features_df['average_negative_cash_balance'] = account_features_df['average_negative_cash_balance'].fillna(0)



monthly_cash_balances['is_negative'] = monthly_cash_balances['running_cash_balance'] < 0
monthly_cash_balances['group'] = (monthly_cash_balances['is_negative'] != 
                                 monthly_cash_balances['is_negative'].shift()).cumsum()


negative_streaks = monthly_cash_balances[monthly_cash_balances['is_negative']].groupby(
    ['current_account_nbr', 'group']).size().reset_index()
negative_streaks.columns = ['current_account_nbr', 'group', 'streak_duration']


max_negative_duration = negative_streaks.groupby('current_account_nbr')['streak_duration'].max().reset_index()
max_negative_duration.columns = ['current_account_nbr', 'max_negative_cash_balance_duration']

account_features_df = account_features_df.merge(max_negative_duration, on='current_account_nbr', how='left')
account_features_df['max_negative_cash_balance_duration'] = account_features_df['max_negative_cash_balance_duration'].fillna(0)


account_features_df.drop(columns=['transaction_amt'], inplace=True)

account_features_df['cash_balance_pct_credit_line'] = (account_features_df['cash_balance'] / account_features_df['cu_crd_line']) * 100




account_features_df['customer_lifecycle_stage'] = pd.cut(
    account_features_df['months_on_books'], 
    bins=[0, 3, 12, float('inf')],
    labels=['New', 'Established', 'Mature']
)


account_features_df['customer_lifetime_value'] = (
    account_features_df['total_sales_last_6_months'] * 
    (1 + account_features_df['months_on_books'] / 12)
)


def analyze_payment_trend(payment_hist):
    if not isinstance(payment_hist, str) or len(payment_hist) < 3:
        return 0, 0, 0, 0
    
    delinq_map = {
        'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7,
        'i': 0, 'j': 1, 'k': 2, 'l': 3, 'm': 4, 'n': 5, 'o': 6, 'p': 7,
        'u': 4, 'v': 5, 'w': 6, 'x': 7,
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
        'z': 0, '%': 0, '#': 0, '+': 0, '-': 0, 'q': 0
    }
    
    
    first_3_months = payment_hist[:3].lower()
    last_3_months = payment_hist[-3:].lower()
    
    first_3_delinq = [delinq_map.get(c, 0) for c in first_3_months]
    last_3_delinq = [delinq_map.get(c, 0) for c in last_3_months]
    
    
    first_3_avg = sum(first_3_delinq) / len(first_3_delinq) if first_3_delinq else 0
    last_3_avg = sum(last_3_delinq) / len(last_3_delinq) if last_3_delinq else 0
    
    
    delinquency_trend = last_3_avg - first_3_avg
    
    
    on_time_count = sum(1 for c in payment_hist.lower() if c in 'aiz%#+q-')
    payment_consistency = on_time_count / len(payment_hist)
    
    
    cycles_since_last_delinq = 0
    for i, c in enumerate(reversed(payment_hist.lower())):
        if c in 'bcdefghjklmnopuvwx1234567':
            cycles_since_last_delinq = i
            break
    
    
    delinq_values = [delinq_map.get(c, 0) for c in payment_hist.lower()]
    delinq_diff = [delinq_values[i] - delinq_values[i+1] for i in range(len(delinq_values)-1)]
    delinq_acceleration = sum(delinq_diff) / len(delinq_diff) if delinq_diff else 0
    
    return delinquency_trend, payment_consistency, cycles_since_last_delinq, delinq_acceleration


payment_trend_features = statement_df.apply(
    lambda row: analyze_payment_trend(row['payment_hist_1_12_mths']), 
    axis=1, 
    result_type='expand'
)
payment_trend_features.columns = [
    'delinquency_trend', 
    'payment_consistency_score', 
    'cycles_since_last_delinquency',
    'delinquency_acceleration'
]
statement_df = pd.concat([statement_df, payment_trend_features], axis=1)


payment_trend_summary = statement_df.groupby('current_account_nbr').agg(
    delinquency_trend=('delinquency_trend', 'mean'),
    payment_consistency_score=('payment_consistency_score', 'mean'),
    cycles_since_last_delinquency=('cycles_since_last_delinquency', 'min'),
    delinquency_acceleration=('delinquency_acceleration', 'mean'),
    
).reset_index()


account_features_df['spending_to_balance_ratio'] = (
    account_features_df['total_sales_last_6_months'] / 
    account_features_df['current_balance']
).replace([np.inf, -np.inf], np.nan).fillna(0)


account_features_df['external_credit_score'] = account_features_df['cu_crd_bureau_scr']


payment_summary['high_return_risk'] = (payment_summary['return_check_count_total'] > 3).astype(int)


account_features_df['weighted_utilization'] = (
    0.5 * account_features_df['utilization_rate'] + 
    0.3 * account_features_df['ca_avg_utilz_lst_3_mnths'] + 
    0.2 * account_features_df['ca_avg_utilz_lst_6_mnths']
)


account_features_df['credit_limit_utilization_trend'] = np.where(
    account_features_df['months_since_credit_limit_change'] > 0,
    (account_features_df['utilization_rate'] - account_features_df['ca_avg_utilz_lst_6_mnths']) / 
    account_features_df['months_since_credit_limit_change'],
    0
)


account_features_df['income_stability_score'] = 1 / (1 + account_features_df['sales_volatility'])
account_features_df['income_stability_score'] = account_features_df['income_stability_score'].fillna(0)


account_features_df['cash_buffer_ratio'] = account_features_df['cash_balance'] / account_features_df['avg_monthly_sales_last_6_months']
account_features_df['cash_buffer_ratio'] = account_features_df['cash_buffer_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)


final_data = account_info_df.copy()
final_data = final_data.merge(spending_pivot, on='current_account_nbr', how='left')
final_data = final_data.merge(payment_summary, on='current_account_nbr', how='left')
final_data = final_data.merge(payment_trend_summary, on='current_account_nbr', how='left')
final_data = final_data.merge(fraud_case_summary, left_on='current_account_nbr', right_on='current_account_nbr', how='left')
final_data = final_data.merge(latest_transaction[['current_account_nbr', 'days_since_last_transaction']], on='current_account_nbr', how='left')

final_data ['payment_full_consistency_score'] = (
    final_data ['payment_consistency_score'] * 
    (1 + final_data ['zero_balance_count'] / 12)
)

account_features_columns = [
    'current_account_nbr', 'utilization_rate', 'open_to_buy', 'current_balance',
    'max_days_delinquent', 'current_cycles_delinquent', 'balance_to_credit_ratio',
    'cash_balance_to_cash_line_ratio', 'credit_grade_numeric', 'months_since_active',
    'months_since_credit_limit_change', 'utilization_stability', 'financial_stress_indicator',
    'months_on_books', 'total_sales_last_6_months', 'avg_monthly_sales_last_6_months',
    'sales_trend', 'nsf_count_last_12_months', 'behavior_score_change',
    'behavior_score_percent_change', 'high_delinquency_flag', 'rapid_balance_change_flag',
    'fraud_risk_score', 'sales_volatility', 'customer_lifecycle_stage', 'customer_lifetime_value',
    'spending_to_balance_ratio', 'external_credit_score', 'weighted_utilization',
    'credit_limit_utilization_trend', 'income_stability_score', 'cash_balance',
    'cash_balance_pct_credit_line', 'cash_buffer_ratio'
]

account_features_columns.extend([
    'negative_cash_balance_flag',
    'cash_balance_volatility',
    'cash_flow_efficiency',
    'cash_balance_trend',
    'cash_to_debt_ratio',
    'cash_buffer_ratio',
    'negative_cash_balance_frequency',
    'average_negative_cash_balance',
    'max_negative_cash_balance_duration'
])



for i in range(1, 6):
    account_features_columns.append(f'sales_change_month_{i}_to_{i+1}')

final_data = final_data.merge(
    account_features_df[account_features_columns], 
    on='current_account_nbr', 
    how='left'
)


final_data.fillna({
    'gross_fraud_amt': 0,
    'net_fraud_amt': 0,
    'fraud_frequency': 0,
    'fraud_ratio': 0,
    'delinquency_count': 0,
    'zero_balance_count': 0,
    'overlimit_count': 0,
    'return_check_count_total': 0,
    'max_days_delinquent': 0,
    'current_cycles_delinquent': 0,
    'nsf_count_last_12_months': 0,
    'fraud_risk_score': 0,
    'delinquency_trend': 0,
    'payment_consistency_score': 0,
    'cycles_since_last_delinquency': 12,  
    'delinquency_acceleration': 0,
    'high_return_risk': 0,
    'weighted_utilization': 0,
    'credit_limit_utilization_trend': 0,
    'income_stability_score': 0,
    'payment_full_consistency_score': 0,
    'cash_balance': 0,
    'cash_balance_pct_credit_line': 0,
    'cash_buffer_ratio': 0
}, inplace=True)

final_data.fillna({
    'negative_cash_balance_flag': 0,
    'cash_balance_volatility': 0,
    'cash_flow_efficiency': 0,
    'cash_balance_trend': 0,
    'cash_to_debt_ratio': 0,
    'cash_buffer_ratio': 0,
    'negative_cash_balance_frequency': 0,
    'average_negative_cash_balance': 0,
    'max_negative_cash_balance_duration': 0
}, inplace=True)

threshold = int(0.2 * final_data.shape[1])  
final_data.dropna(thresh=threshold, inplace=True)


final_data.to_csv("enriched_account_information.csv", index=False)

### Code for EDA

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def perform_eda(df, file_name):
    print(f"Performing EDA for {file_name}...")
    
    
    empty_columns = [col for col in df.columns if df[col].isnull().all() or (df[col].astype(str).str.strip() == '').all()]
    if empty_columns:
        print(f"Dropping columns with all empty values: {empty_columns}")
        df = df.drop(columns=empty_columns)
    
    
    constant_columns = [col for col in df.columns if df[col].nunique() == 1]
    if constant_columns:
        print(f"Dropping columns with all identical values: {constant_columns}")
        df = df.drop(columns=constant_columns)
    
    print("Shape of the data after dropping empty and constant-value columns:", df.shape)
    print("Columns:", df.columns)
    print("Data Types:\n", df.dtypes)
    
    
    missing_values = df.isnull().sum()
    print("Missing Values:\n", missing_values[missing_values > 0])
    
    
    numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = df.select_dtypes(include=['object']).columns.tolist()
    
    
    date_features = []
    for col in categorical_features:
        try:
            pd.to_datetime(df[col], errors='raise')  
            date_features.append(col)
        except Exception:
            pass  
    
    
    if numerical_features:
        print("\nNumerical Feature Analysis:")
        print(df[numerical_features].describe())
        
        
        corr_matrix = df[numerical_features].corr()
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title(f"Correlation Matrix for {file_name}")
        plt.show()
        
        
        for col in numerical_features:
            print(f"\nStatistics for {col}:")
            print(f"Mean: {df[col].mean():.2f}, Median: {df[col].median():.2f}, Min: {df[col].min()}, Max: {df[col].max()}, Std: {df[col].std():.2f}")
            
            
            plt.figure(figsize=(10, 5))
            sns.histplot(df[col], kde=True)
            plt.title(f"Distribution of {col}")
            plt.show()
            
            sns.boxplot(x=df[col])
            plt.title(f"Boxplot of {col}")
            plt.show()
    
    
    if categorical_features:
        print("\nSkipping Categorical Feature Analysis.")
    
    
    if date_features:
        print("\nDate Feature Analysis:")
        for col in date_features:
            try:
                
                df[col] = pd.to_datetime(df[col], errors='coerce')
                
                
                valid_dates = df[col].dropna()
                invalid_count = df[col].isna().sum()
                
                if invalid_count > 0:
                    print(f"Found {invalid_count} invalid dates in {col}")
                    
                if not valid_dates.empty:
                    print(f"\nDate Range for {col}:")
                    print(f"Min Date: {valid_dates.min()}, Max Date: {valid_dates.max()}")
                    
                    
                    if valid_dates.dt.tz is not None:
                        valid_dates = valid_dates.dt.tz_localize(None)
                    
                    
                    date_range = valid_dates.dt.to_period('M').value_counts().sort_index()
                    
                    
                    plt.figure(figsize=(12, 6))
                    date_range.plot(kind='bar')
                    plt.title(f"Monthly Trend Analysis of {col}")
                    plt.xlabel("Month")
                    plt.ylabel("Frequency")
                    plt.xticks(rotation=45)
                    plt.show()
                else:
                    print(f"No valid dates found in column {col}")
                    
            except Exception as e:
                print(f"Error processing date column {col}: {e}")

def process_all_csvs(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path)
            perform_eda(df, file)
        except Exception as e:
            print(f"Error processing {file}: {e}")


data_folder = "data"
process_all_csvs(data_folder)


def process_all_csvs(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path)
            perform_eda(df, file)
        except Exception as e:
            print(f"Error processing {file}: {e}")


data_folder = "data"
process_all_csvs(data_folder)